With Monte-Carlo method it is necessary to wait until the end of the episode before updating the value function. This is a serious problem because some applications can have very long episodes with learning delayed to the end of each one. Moreover, in some environments the completion of the episode is not guaranteed. Hence to solve this problem we'll use **Temporal-Difference method**.

#### Temporal-Difference(0) based Prediction

In [1]:
import os
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
np.set_printoptions(suppress=True)  # to aviod scientific notation while printing numpy array
sys.path.append('..')

from environment import Gridworld

env = Gridworld()
gamma = 0.999 # discount factor
alpha = 0.01

V = np.zeros((3,4)) # Defining an empty state-value matrix

# INPUT -> Optimal policy using value iteration, {up=0, left=1, down=2, right=3}
policy = np.array([[3,      3, 3, -1],
                   [0, np.NaN, 0, -1],
                   [0,      1, 1,  1]])

tot_episodes = 50000 # no of eposides
time_step_for_task = 20 # time steps required to perform a task in one episode

for current_episode in range(tot_episodes):
    robot_current_state = env.reset() # Reset and return the first observation

    # Now we can run for some time step within the episode
    for step_time in range(time_step_for_task): 
        action = policy[robot_current_state[0], robot_current_state[1]]
        action_applied = np.random.choice(4, 1, p=env.action_transition_matrix[int(action),:]) # Generate a non-uniform random sample (following p-distribution) from np.arange(4) of size 1
        
        next_position = env.step(action) # perform this action
        
        state_reward = env.reward[next_position[0], next_position[1]] # state reward obtained (for state at t+1)

        # One-step TD(0) update
        V_current_xk = V[robot_current_state[0], robot_current_state[1]] # V(x_{k}) -> current
        V_current_xk1 = V[next_position[0], next_position[1]] # V(x_{k+1}) -> current
        rk1 = state_reward # r_{k+1}

        V[robot_current_state[0],robot_current_state[1]] = V_current_xk + alpha*(rk1 + gamma*V_current_xk1 - V_current_xk) # V(x_{k}) -> new

        robot_current_state = next_position # Update robot current position (to observation at t+1)

        if env.state_matrix[next_position[0], next_position[1]] == 1:
            break

    
V[np.where(env.reward == 1)] = 1
V[np.where(env.reward == -1)] = -1

# Time to check the utility matrix obtained
print("State-Value matrix after " + str(tot_episodes) + " iterations:")
print(V)

State-Value matrix after 50000 iterations:
[[ 0.84578261  0.90616525  0.95581157  1.        ]
 [ 0.796158    0.          0.69817666 -1.        ]
 [ 0.73362195  0.68367526  0.          0.        ]]


#### Temporal-Difference($\lambda$) based Prediction

In [2]:
import os
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
np.set_printoptions(suppress=True)  # to aviod scientific notation while printing numpy array
sys.path.append('..')

from environment import Gridworld

env = Gridworld()
gamma = 0.999 # discount factor
alpha = 0.01
lambda_new = 0.5

V = np.zeros((3,4)) # Defining an empty state-value matrix

#Define and print the eligibility trace matrix
trace_matrix = np.zeros((3,4))

# INPUT -> Optimal policy using value iteration, {up=0, left=1, down=2, right=3}
policy = np.array([[3,      3, 3, -1],
                   [0, np.NaN, 0, -1],
                   [0,      1, 1,  1]])

tot_episodes = 50000 # no of eposides
time_step_for_task = 1000 # time steps required to perform a task in one episode

for current_episode in range(tot_episodes):
    robot_current_state = env.reset() # Reset and return the first observation

    # Now we can run for some time step within the episode
    for step_time in range(time_step_for_task): 
        action = policy[robot_current_state[0], robot_current_state[1]]

        next_position = env.step(action) # perform this action
        
        state_reward = env.reward[next_position[0], next_position[1]] # state reward obtained (for state at t+1) -> r_{k+1}

        # One-step TD(lambda) update
        V_current_xk = V[robot_current_state[0], robot_current_state[1]] # V(x_{k}) -> current
        V_current_xk1 = V[next_position[0], next_position[1]] # V(x_{k+1}) -> current
        
        delta = state_reward + gamma*V_current_xk1 - V_current_xk # TD error

        trace_matrix[robot_current_state[0], robot_current_state[1]] += 1 # Adding +1 in the trace matrix (only the state visited)

        V += alpha*delta*trace_matrix # V(x_{k}) -> new

        trace_matrix = gamma*lambda_new*trace_matrix # updating trace_matrix

        robot_current_state = next_position # Update robot current position (to observation at t+1)

        if env.state_matrix[next_position[0], next_position[1]] == 1:
            break

V[np.where(env.reward == 1)] = 1
V[np.where(env.reward == -1)] = -1

# Time to check the utility matrix obtained
print("State-Value matrix after " + str(tot_episodes) + " iterations:")
print(V) 

State-Value matrix after 50000 iterations:
[[ 0.86346025  0.92387343  0.97134743  1.        ]
 [ 0.81242462  0.          0.78939655 -1.        ]
 [ 0.75046207  0.68465266  0.          0.        ]]
